In [46]:
#Customer LifeTime Value (müşteri yaşam boyu değeri)

# 1. Veri Hazırlama
# 2. Average Order Value (ortalama sipariş tutarı) = (average order value = total (price)revenue / total number of orders(total satış geliri / toplam sipariş sayısı))
# 3. Purchase Frequency (satın alma sıklığı) = (total number of orders(transiction) / total number of customers) (toplam sipariş sayısı / toplam müşteri sayısı)
# 4. Repeat Rate (tekrarlanma oranı) = (total number of repeat orders(transiction) / total number of orders(transiction)) (toplam tekrarlanan sipariş sayısı / toplam sipariş sayısı)
# 5. Profit Margin (kazanç oranı) = (total (price)revenue / total number of orders(total satış geliri / toplam sipariş sayısı))
# 6. Customer Value (müşteri değeri) = (average order value * purchase frequency)
# 7. Customer Lifetime Value (müşteri yaşam boyu değeri) = (customer value / churn rate) x profit margin
# 8. Segmentation (müşteri segmentasyonu)



In [47]:
# Veri setindeki değişkenlerin tanımları:
# InvoiceNo: Fatura numarası. Her işleme yani faturaya ait eşsiz numara. C ile başlıyorsa iptal edilen işlem.
# StockCode: Ürün kodu. Her bir ürün için eşsiz numara.
# Description: Ürün ismi
# Quantity: Ürün adedi. Faturalardaki ürünlerin adedi.
# InvoiceDate: Fatura tarihi ve zamanı.
# UnitPrice: Ürün fiyatı (Sterlin cinsinden)
# CustomerID: Eşsiz müşteri numarası
# Country: Ülke ismi. Müşterinin yaşadığı ülke.

In [48]:
import pandas as pd 
from sklearn.preprocessing import MinMaxScaler
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option("display.float_format", lambda x: "%.2f" % x)

In [49]:

df_ = pd.read_excel("online_retail_II.xlsx", sheet_name="Year 2009-2010")


In [50]:
df = df_.copy()


In [51]:

df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.00,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.00,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.00,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.00,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.00,United Kingdom


In [52]:
df.isnull().sum()

Invoice             0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
Price               0
Customer ID    107927
Country             0
dtype: int64

In [56]:
df = df[~df["Invoice"].str.contains("C", na= False)]
df.head(5)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.00,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.00,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.00,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.00,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.00,United Kingdom


In [60]:
df.describe().T

,count,mean,min,25%,50%,75%,max,std
Quantity,515255.00,10.96,-9600.00,1.00,3.00,10.00,19152.00,104.35
InvoiceDate,515255,2010-06-28 17:41:06.558422528,2009-12-01 07:45:00,2010-03-21 13:27:00,2010-07-06 13:13:00,2010-10-15 14:27:00,2010-12-09 20:01:00,NaN
Price,515255.00,3.96,-53594.36,1.25,2.10,4.21,25111.09,127.69
Customer ID,407695.00,15368.50,12346.00,13997.00,15321.00,16812.00,18287.00,1679.80


In [61]:
df = df[(df['Quantity'] > 0)]

In [62]:
df.describe().T

,count,mean,min,25%,50%,75%,max,std
Quantity,513134.00,11.72,1.00,1.00,3.00,10.00,19152.00,92.97
InvoiceDate,513134,2010-06-28 21:20:02.384289536,2009-12-01 07:45:00,2010-03-21 15:03:00,2010-07-06 16:26:00,2010-10-15 14:51:00,2010-12-09 20:01:00,NaN
Price,513134.00,3.97,-53594.36,1.25,2.10,4.21,25111.09,127.95
Customer ID,407695.00,15368.50,12346.00,13997.00,15321.00,16812.00,18287.00,1679.80


In [63]:
df.dropna(inplace=True)

In [64]:
df["TotalPrice"] = df["Quantity"] * df["Price"]  



In [66]:
cltv_c = df.groupby('Customer ID').agg({'Invoice': lambda x: x.nunique(),
                                        'Quantity': lambda x: x.sum(),
                                        'TotalPrice': lambda x: x.sum()
                                        })   

cltv_c.columns = ['total_transaction', 'total_unit', 'total_price']

print(cltv_c.head()) #müşteri bazında toplam işlem, toplam ürün ve toplam fiyat bilgileri

             total_transaction  total_unit  total_price
Customer ID                                            
12346.00                    11          70       372.86
12347.00                     2         828      1323.32
12348.00                     1         373       222.16
12349.00                     3         993      2671.14
12351.00                     1         261       300.93


In [67]:
# 2. Average Order Value (ortalama sipariş tutarı) = (average order value = total (price)revenue / total number of orders(total satış geliri / toplam sipariş sayısı))
cltv_c["avg_order_value"] = cltv_c["total_price"] / cltv_c["total_transaction"] 

# 3. Purchase Frequency (sipariş frekansı) = total number of orders(total satış geliri / toplam sipariş sayısı)

In [68]:
cltv_c.head()

,total_transaction,total_unit,total_price,avg_order_value
Customer ID,,,,
12346.00,11,70,372.86,33.90
12347.00,2,828,1323.32,661.66
12348.00,1,373,222.16,222.16
12349.00,3,993,2671.14,890.38
12351.00,1,261,300.93,300.93


3. Purchase Frequency (satın alma sıklığı) = (total number of orders(transiction) / total number of customers) (toplam sipariş sayısı / toplam müşteri sayısı)
 

In [83]:
cltv_c["purchase_frequency"] = cltv_c["total_transaction"] / cltv_c.shape[0] #toplam işlem sayısı / toplam müşteri sayısı
# bu datırda cltv_c["purchase_frequency"] değişkenini oluşturduk.
# bütün müşterilerin işlem sayısının toplamını aldık.

4. Repeat Rate (tekrarlanma oranı) = (total number of repeat orders(transiction) / total number of orders(transiction)) (toplam tekrarlanan sipariş sayısı / toplam sipariş sayısı)


In [71]:
repet_rate = cltv_c[cltv_c["total_transaction"] > 1].shape[0] / cltv_c.shape[0] # 1'den fazla işlem yapan müşteri sayısı / toplam müşteri sayısı"]]
# 1'den fazla işlem yapan müşteri sayısı / toplam müşteri sayısı  bu değer bizim repeat rate'imizdir.


In [72]:
churn_rate = 1 - cltv_c[cltv_c["total_transaction"] > 1].shape[0] / cltv_c.shape[0] # 1'den fazla işlem yapan müşteri sayısı / toplam müşteri sayısı

# 5. Profit Margin (kazanç oranı) = (total (price)revenue / total number of orders(total satış geliri / toplam sipariş sayısı))

In [78]:
# kar marjı = 0.10c
cltv_c["profit_margin"] = cltv_c["total_price"] * 0.10
# her bir müşterinin kar marjının ortalaması


#6. Customer Value (müşteri değeri) = (average order value * purchase frequency)

In [84]:
cltv_c["customer_value"] = cltv_c["avg_order_value"] * cltv_c["purchase_frequency"] 
#müşteri değeri = ortalama sipariş tutarı * sipariş frekansı


# 7. Customer Lifetime Value (müşteri yaşam boyu değeri) = (customer value / churn rate) x profit margin

In [85]:
cltv_c.head()

,total_transaction,total_unit,total_price,avg_order_value,profit_margin,purchase_frequency,customer_value
Customer ID,,,,,,,
12346.00,11,70,372.86,33.90,37.29,0.00,0.09
12347.00,2,828,1323.32,661.66,132.33,0.00,0.31
12348.00,1,373,222.16,222.16,22.22,0.00,0.05
12349.00,3,993,2671.14,890.38,267.11,0.00,0.62
12351.00,1,261,300.93,300.93,30.09,0.00,0.07


In [86]:
cltv_c["cltv"] = (cltv_c["customer_value"] / churn_rate) * cltv_c["profit_margin"]

In [87]:
cltv_c.sort_values("cltv", ascending=False).head() #cltv değerine göre sıraladık.

,total_transaction,total_unit,total_price,avg_order_value,profit_margin,purchase_frequency,customer_value,cltv
Customer ID,,,,,,,,
18102.00,89,124216,349164.35,3923.19,34916.44,0.02,80.94,8579573.77
14646.00,78,170342,248396.50,3184.57,24839.65,0.02,57.58,4342070.46
14156.00,102,108107,196566.74,1927.12,19656.67,0.02,45.56,2719105.09
14911.00,205,69722,152147.57,742.18,15214.76,0.05,35.27,1629055.81
13694.00,94,125893,131443.19,1398.33,13144.32,0.02,30.47,1215855.89


# 8. segmentlerin oluşturulması


In [93]:
cltv_c["segment"] = pd.qcut(cltv_c["cltv"], 4, labels=["D", "C", "B", "A"]) #cltv değerlerini 4 eşit parçaya böldük.
cltv_c.sort_values(by="cltv", ascending=False).head()

,total_transaction,total_unit,total_price,avg_order_value,profit_margin,purchase_frequency,customer_value,cltv,cltv_segment,segment
Customer ID,,,,,,,,,,
18102.00,89,124216,349164.35,3923.19,34916.44,0.02,80.94,8579573.77,A,A
14646.00,78,170342,248396.50,3184.57,24839.65,0.02,57.58,4342070.46,A,A
14156.00,102,108107,196566.74,1927.12,19656.67,0.02,45.56,2719105.09,A,A
14911.00,205,69722,152147.57,742.18,15214.76,0.05,35.27,1629055.81,A,A
13694.00,94,125893,131443.19,1398.33,13144.32,0.02,30.47,1215855.89,A,A


In [98]:
#cltv_c.groupby("segment").agg({"count", "mean", "sum"})
cltv_c.to_csv("cltv_c.csv")